In [1]:
#SQLAlchemy
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, Integer, String, Float, create_engine, inspect, func
# from sqlalchemy import Column, Integer, String, Float, create_engine, inspect, func
from sqlalchemy.orm import Session

#SQLite 
import sqlite3
from sqlite3 import Error

#Pandas & plt
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import re
import numpy as np

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
engine = create_engine(f"sqlite:///hawaii.sqlite", echo=False)
Base = automap_base()
Base.prepare(engine, reflect = True)
inspector = inspect(engine)

In [3]:
measure = Base.classes.measure_sql

In [4]:
station = Base.classes.station_sql

In [5]:
session = Session(engine)


In [6]:
columns = inspector.get_columns('measure_sql')
for c in columns:
    print (c)

{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'station', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'date', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'prcp', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'tobs', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}


## Precipitation Analysis

In [7]:
mp = session.query(measure.date, measure.prcp).\
    filter(measure.date <= '2017-05-01').\
    filter(measure.date >= '2016-05-01').all()
# mp

In [8]:
conn = engine.connect()
mp = pd.DataFrame(mp, columns=["Date", "Prcp"])
mp.head()

,Date,Prcp
0,2016-05-01,0.00
1,2016-05-02,0.00
2,2016-05-03,0.01
3,2016-05-04,0.00
4,2016-05-05,0.13


In [9]:
# mp = mp.set_index('Date')

In [10]:
mp = mp.groupby('Date').mean()

In [11]:
# mp['index'] = range(1, len(mp) + 1)
# mp = mp.reset_index('index', drop = False)
# mp.head()

In [12]:
mp['Prcp'].plot()
labels = ['May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'January', 'February', 'March','April']
locs = [0,30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360]
plt.xticks(locs, labels, rotation=20)
plt.xticks(rotation='45')
plt.tight_layout()
plt.show

<function matplotlib.pyplot.show>

In [13]:
# plot.set_xticklabels(mp.index)

In [14]:
# list(np.ravel)

## Station Analysis

In [15]:
total_stations = session.query(station.station).count()
print(f"The Total Stations Collecting data is {total_stations}")

The Total Stations Collecting data is 9


In [16]:
ts = session.query(measure.station, measure.date).all()
# ts

In [17]:
station_count = pd.DataFrame(ts)
group_station = station_count.groupby('station').count()
sort_station = group_station.sort_values('date', ascending=False, inplace=False)
sort_station

,date
station,
USC00519281,2772
USC00513117,2696
USC00519397,2685
USC00519523,2572
USC00516128,2484
USC00514830,1937
USC00511918,1932
USC00517948,683
USC00518838,342


USC00519281 has the highest number of observations.

In [18]:
tobs_stations = session.query(measure.date, measure.station, measure.tobs).\
    filter(measure.date <= '2017-05-01').\
    filter(measure.date >= '2016-05-01').\
    filter(measure.station == 'USC00519281').all()
tobs_stations

[('2016-05-01', 'USC00519281', 75.0),
 ('2016-05-02', 'USC00519281', 73.0),
 ('2016-05-03', 'USC00519281', 74.0),
 ('2016-05-04', 'USC00519281', 72.0),
 ('2016-05-05', 'USC00519281', 68.0),
 ('2016-05-06', 'USC00519281', 66.0),
 ('2016-05-07', 'USC00519281', 69.0),
 ('2016-05-08', 'USC00519281', 67.0),
 ('2016-05-09', 'USC00519281', 68.0),
 ('2016-05-10', 'USC00519281', 68.0),
 ('2016-05-11', 'USC00519281', 67.0),
 ('2016-05-12', 'USC00519281', 65.0),
 ('2016-05-13', 'USC00519281', 68.0),
 ('2016-05-14', 'USC00519281', 70.0),
 ('2016-05-15', 'USC00519281', 76.0),
 ('2016-05-16', 'USC00519281', 75.0),
 ('2016-05-17', 'USC00519281', 75.0),
 ('2016-05-18', 'USC00519281', 72.0),
 ('2016-05-19', 'USC00519281', 75.0),
 ('2016-05-20', 'USC00519281', 72.0),
 ('2016-05-21', 'USC00519281', 76.0),
 ('2016-05-22', 'USC00519281', 76.0),
 ('2016-05-23', 'USC00519281', 74.0),
 ('2016-05-24', 'USC00519281', 69.0),
 ('2016-05-25', 'USC00519281', 71.0),
 ('2016-05-26', 'USC00519281', 74.0),
 ('2016-05-2

In [19]:
USC00519281 = pd.DataFrame(tobs_stations)
USC00519281.head()

,date,station,tobs
0,2016-05-01,USC00519281,75.0
1,2016-05-02,USC00519281,73.0
2,2016-05-03,USC00519281,74.0
3,2016-05-04,USC00519281,72.0
4,2016-05-05,USC00519281,68.0


In [20]:
group_tobs = USC00519281.sort_values(['tobs', 'date'], ascending=False).groupby('tobs').count()
group_tobs.head()

,date,station
tobs,,
59.0,3,3
60.0,2,2
61.0,1,1
62.0,6,6
63.0,4,4


In [21]:
# bins = [57.5, 60, 62.5, 65, 67.5, 70, 72.5, 75, 77.5, 80, 82.5, 85]
# plt.bar(group_tobs.index, group_tobs.date)
# plt.xlabel('Temperature')
# plt.ylabel('Frequency')

In [22]:
plt.hist(USC00519281.tobs, bins = 12)
plt.xlabel('Temperature')
plt.ylabel('Frequency')

Text(36.625,0.5,'Frequency')

## Temperature Analysis

In [23]:
def calc_temps(start_date, end_date):
    if re.match(r'[0-9]{4}-[0-9]{2}-[0-9]{2}$', start_date) and re.match(r'[0-9]{4}-[0-9]{2}-[0-9]{2}$', end_date):
        return session.query(func.min(measure.tobs), func.max(measure.tobs), func.avg(measure.tobs)).\
        filter(measure.date <= end_date).\
        filter(measure.date >= start_date).all()[0]
    else:
        return 'Invalid date format, please enter again'

In [ ]:
start_date = input("Please enter a start date in the format of YYYY-MM-DD")
end_date = input("Please enter a end date in the format of YYYY-MM-DD")    

In [25]:
tmin, tmax, tavg = calc_temps(start_date, end_date)

ValueError: too many values to unpack (expected 3)

In [ ]:
plt.boxplot([tmin, tmax, tavg])

In [ ]:
tmin, tmax, tavg